# Week 9: Networks 1


A network is a way to represent a system of things that interact or are interrelated in some fashion. It could just be your facebook network, that's obvious, but a network can also represent the interaction patterns of proteins in the human body, who votes what on whose bill in the US Senate, the mutually attractive forces between galaxies in the universe or just about anything else that you can possibly imagine. This week we will get started on some of the fundamental concepts, and then look closer at some Facebook data. In the exercises today you will:

* Get warmed up to thinking about systems as networks
* Learn to use the `NetworkX` library
* Explore a Facebook network

# Lecture

In [ ]:
# ----------------- !
# Execute this cell !
# ----------------- !

from IPython.display import HTML

def video_html(url):
    return f"""
    <div align="middle">
    <video width="50%" controls>
      <source src="{url}" type="video/mp4">
    </video></div>"""

HTML(video_html('https://www.dropbox.com/s/43ya0a9jmaa6exk/slides_week9_mov.m4v?raw=1'))

## Imports

In [3]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import networkx as nx           # pip install networkx
import pandas as pd
import seaborn as sns
from collections import Counter
from netwulf import visualize   # pip install netwulf

## Exercises

### Part 0: Networks

Questions for chapter 1 and 2 of [Network Science](http://barabasi.com/networksciencebook/).

> **Ex. 9.0.1**: List three different real networks and state the nodes and links for each of them.

##### Answer Ex. 9.0.1:
| Network                 | Nodes          | Links       |
|-------------------------|----------------|-------------|
| Foodweb                 | Ingredients    | Recipes     |
| Railroad transportation | Train stations | Railroad    |
| Twitter Hashtags        | Hashtags       | Tweets      |

> **Ex. 9.0.2**: What networks are you personally most interested in. Address the following questions:
>   * What are its nodes and links? 
>   * How large is it? 
>   * Can be mapped out? 
>   * Why do you care about it? 

##### Answer to Ex 9.0.2.:
*Personally I am mostly interested in non-trivial networks. That is, a system which is not traditionally considered a network with nodes and links. A silly example could be the food recipes in 'Nak & Æd'. The nodes are the recipes and the links are all the ingredients. This is not a comprehensive network, which make it feasible for mapping. The mapping of the network reveaæs that butter and oil is a very central element (node if you will) in Nikolajs recipes. I care about it because its a silly example and I enjoy the TV show.* 

> **Ex. 9.0.3**: In your view what would be the area where network science could have the biggest impact in the next decade? Explain your answer - and base it on the text in the book. 

##### Answer to Ex.9.0.3:
*An obvious candidate would be social networks and online behavior in general. For one, we have an exponential growing digital record of social connections and our improving computitional power allows us to empirically apply network theory on these data sources. One simply needs to open any homepage on the www to see how google applies network theory to customize user-specific ads. For example, they show ads of products that people with similar browser history have purchased (or at least visited the webshop).*

> **Ex. 9.0.4**: Section 2.5 states that real networks are sparse. Can you think of a real network where each node has *many* connections? Is that network still sparse? If yes, can you explain why?

##### Answer to Ex. 9.0.4:
*Honestly, I have trouble thinking of a sparse real network. In regards to a real network with many connections, the network is not necessarily a necessarily a spare network. The spacity relates to the relative number of connections compared to the total nodes (the potential number of nodes) in the network. For example, most Facebook users have more than 100+ connections on Facebook. However, considering the fact that Facebook has +2 billion users, 100 connections is a very small number.*

### Part 1: The `NetworkX` library

`NetworkX` is the Python library we will be using for working with networks. It should come installed with your Anaconda distribution, but you wont know how to use it yet. The best way to get familiar is to work through a tutorial. That's what the following exercises are about.

> **Ex. 9.1.1**: Go to the `NetworkX` project's [tutorial page](https://networkx.github.io/documentation/stable/tutorial.html). Read the first few sections about nodes, edges and attributes. Write an example below where you create a graph, `G`, and then:
1. add some nodes and edges to,
2. print its nodes and edges
3. delete a node, and
4. plot it.

In [ ]:
# Answer to Ex 9.1.1.:
import networkx as nx

# Create a graph (network):
G = nx.Graph()

# Add 3 nodes to graph with id 1,2,3
G.add_nodes_from([1,2,3])

## Add edges between the three nodes
G.add_edges_from([(1,2),(1,3)])

## Remove node 2
G.remove_node(2)

## Draw graph
nx.draw(G,with_labels=True,font_weight='bold')

> **Ex. 9.1.2**: `NetworkX` has lots of functions to generate synthetic networks. One of the most famous types of synthetic networks is called an Erdős–Rényi (ER) network.
1. Describe what an ER network is and how to create one.
2. On the [`NetworkX` documentations site](https://networkx.github.io/documentation/networkx-1.10/reference/generators.html), find a method you can use for generating an ER network. Create one with $N=100$ nodes and probability of edge creation $p=0.1$.
3. Plot the network
4. Visualize the [degree distribution](https://en.wikipedia.org/wiki/Degree_distribution) of your newly created ER graph.
    * A node's *degree* is the number of other nodes it is connected to.
    * The *degree distribution* is a histogram which shows the distribution of node degrees.

##### Answer Ex. 9.1.2.:
An ER network is a randomized network with N nodes. The edges between each node has a fixed probability of occuring. Moreover, these edge probabilities are independent of each other, meaning that the creation of edge B won't affect the probability of edge C to exists. If the network is unweighted, that is binary edges, it's simply a question of making draws from a benioulli distribution among all possible edges. 

In [ ]:
# Create ER network
G_ER = nx.fast_gnp_random_graph(n=100,p=0.1,seed=1337)

# Draw graph
nx.draw(G_ER,with_labels=False)
plt.title('Random generation of an ER network')

In [ ]:
# Draw histogram of degrees in ER graph
y = nx.degree_histogram(G_ER)
x = list(range(len(y)))
fig,ax = plt.subplots()
plt.bar(x,y)
ax.set_xticks(x)
ax.set_xlabel('Degrees')
ax.set_ylabel('# of nodes')
ax.set_title('Degree distribution in a randomized ER network')
plt.show()

> **Ex. 9.1.3**: Another very famous type of synthetic network is the Barabási–Albert (BA) network.
1. Describe what a BA network is and how to create one.
2. On the [`NetworkX` documentations site](https://networkx.github.io/documentation/networkx-1.10/reference/generators.html), find a method you can use for generating a BA network. Create one with $N=100$ nodes and number of new edges from each new node $m=2$.
3. Plot the network
4. Visualize the [degree distribution](https://en.wikipedia.org/wiki/Degree_distribution) of your newly created BA graph.
5. Explain the main difference(s) you observe, when comparing to the ER network.
6. Are there other measures we could have used to describe the ER and BA networks further?

##### Answer to Ex. 9.1.3.:
According to the documentation in NetworkX, a BA graph is a graph of n nodes is grown by attaching new nodes each with m edges that are preferentially attached to existing nodes with high degree. Hence, the network will have more simularities to observed social networks with cluster nodes which have a high degree (think influencers on Instagram). The degree distibutions of a BA is heavy tailed. In comparision the degree distribution of a ER converges to a normal distribution with the $\mu$ being $N*p$. We may have described the two networks by other network features such as cluster coefficients, avg. degrees, etc. 

In [ ]:
## create BA network
G_BA = nx.barabasi_albert_graph(n=100,m=2,seed=1337)

# Draw histogram of degrees in BA graph
y = nx.degree_histogram(G_BA)
x = list(range(len(y)))
fig,ax = plt.subplots()
plt.bar(x,y)
ax.set_xticks(x)
ax.set_xlabel('Degrees')
ax.set_ylabel('# of nodes')
ax.set_title('Degree distribution in a randomized BA network')
plt.show()

### Part 2: Facebook network

Now that you know how to build a network in `NetworkX`, visualize it and plot some statistics, you are ready to analyze today's dataset. We will be using [this facebook dataset](http://socialnetworks.mpi-sws.org/data-wosn2009.html) collected by the researchers at the Max Planck Institute. Go ahead and download the list of wall posts. Put the file in the same directory as this notebook and load it like:

In [ ]:
data = pd.read_csv(
    "facebook-wall.txt.gz",
    delimiter="\t",
    header=None,
    names=['user1', 'user2', 'timestamp']
)

data.head(10)

As you can see this is a *temporal network* (in that edges are timestamped). Time is given in UNIX timestamps. Though there are ways to model such
a network in its entire complexity, we'll start simple and just take a slice of it out and treat that as a static
network. You can create a `nx.DiGraph` from a slice of time like:

In [ ]:
t0 = data.timestamp.max() - 86400 * 7  # Lower bound of time-slice (here: 7 days before last wall post)
t1 = data.timestamp.max()              # Upper bound of time-slice

# Subset of the pandas dataframe
data_t = data.loc[data.timestamp.between(t0, t1)]

# Count the number of times each link occurs and store that in a new 'weight' column
data_t = data_t.groupby(['user1', 'user2']).size().reset_index(name='weight')

# Create a `nx.DiGraph` from this dataframe
G_t = nx.from_pandas_edgelist(data_t, 'user1', 'user2', 'weight', create_using=nx.DiGraph)

> **Ex: 9.2.1**: Convert the above sample code into a function where you can easily input `t0` and `t1` of the slice, and get a `nx.DiGraph` in return. Create the aggregated network for the last 365 days of the data. Print the number of nodes and edges.

In [ ]:
def ans_ex_9_2_1(df_wall_posts,lag_days):
    """
    arguments:
    df_wall_posts: pandas df
    - contains user1, user2 and unix timestamp
    
    lag_days: int
    - the lag of days to include in the nx graph
    
    returns:
    G_t: nx.DiGraph
    - a graph with weights within the specified time range
    """
    t0 = data.timestamp.max() - 86400 * lag_days  # Lower bound of time-slice (here: 7 days before last wall post)
    t1 = data.timestamp.max()                     # Upper bound of time-slice

    # Subset of the pandas dataframe
    data_t = data.loc[data.timestamp.between(t0, t1)]

    # Count the number of times each link occurs and store that in a new 'weight' column
    data_t = data_t.groupby(['user1', 'user2']).size().reset_index(name='weight')

    # Create a `nx.DiGraph` from this dataframe
    G_t = nx.from_pandas_edgelist(data_t, 'user1', 'user2', 'weight', create_using=nx.DiGraph)
    
    return G_t    

In [ ]:
G_t_365 = ans_ex_9_2_1(df_wall_posts=data,lag_days=365)
print(f'Number of nodes in 365 days network: {G_t_365.number_of_nodes()}')
print(f'Number of edges in 365 days network: {G_t_365.number_of_edges()}')

> **Ex: 9.2.2**: Since it's a directed network, there are two degree distributions we can plot: one for the
in-degrees and another for the out-degrees. Plot them both in the same figure (remember to add a legend).
The distribution is very heavy-tailed, so you may want to log-scale both axes. Comment on the result. Does
it resemble any of the network models you know?
>
> *Hint: use `Counter` to count how many times each degree-value occurs and then scatter plot the degree-values
against counts. When you log-scale the axes, zero values cause trouble, so you should remove the data point for
degree 0.*

In [ ]:
df_in_degrees = pd.DataFrame({'node_id':[n for n,d in G_t_365.in_degree()],
                          'in_degrees':[d for n,d in G_t_365.in_degree()]
                          })
df_out_degrees = pd.DataFrame({'node_id':[n for n,d in G_t_365.out_degree()],
                          'out_degrees':[d for n,d in G_t_365.out_degree()]
                          })

df_degrees  = df_in_degrees\
.merge(df_out_degrees,on='node_id',how='left')\
.set_index('node_id')\
.stack()\
.reset_index()\
.sort_values(by='node_id')\
.set_index('node_id')\
.rename(columns={'level_1':'degree_type',0:'degrees'})\
.assign(no_of_nodes = 1)\
.groupby(['degree_type','degrees']).count()\
.reset_index()\
.assign(log_no_of_obs = lambda df:np.log(df.no_of_nodes))\
.assign(log_degrees = lambda df:np.log(df.degrees))


ax = sns.scatterplot(x='degrees',y='no_of_nodes',hue='degree_type',data=df_degrees)
plt.title('Degree distribution')
plt.ylabel('# of nodes')
plt.show()

In [ ]:
ax = sns.scatterplot(x='log_degrees',y='log_no_of_obs',hue='degree_type',data=df_degrees)
plt.title('Degree distribution\n(log transformed)')
plt.ylabel('# of nodes')
plt.show()

> **Ex: 9.2.3**: Using a histogram plot the distribution of (weighted) local clustering coefficients ([docs](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.cluster.clustering.html#networkx.algorithms.cluster.clustering)). Which distribution does it appear to follow?
>
> *Hint: figure out how to plot a histogram with logarithmic bins.*

##### Answer to Ex 9.2.3.:

In [ ]:
## Calculate cluster bins with weights
cluster_coefs        = nx.clustering(G_t_365,weight='weight')

## Create hist plot with log bins
min_value, max_value = sorted(set(cluster_coefs.values()))[1], sorted(set(cluster_coefs.values()))[-1]
bins                 = np.logspace(np.log10(min_value), np.log10(max_value), 100)
plt.hist(cluster_coefs.values(), bins = bins)
plt.xscale("log")
plt.title('Log distribution of weighted local clustering coefs')
plt.show()

We want to plot the network so we can visually get a feeling for how people are connected. `networkx`'s `nx.draw` method won't cut it here, it's too slow and has bad layout support for large networks, so instead we'll use `netwulf` ([docs](https://netwulf.readthedocs.io/)) to visualize networks in a browser-app I made (still in beta, bugs may occur). But even with this tool, the network of one year is still too big to make nice visualizations, so we have to reduce the number of links and nodes by some heuristic. This could for example be to threshold link weight to reduce the number of links, thresholding weighted node degree (or other relevant metrics).

> **Ex: 9.2.4**: Threshold the network by some meaningful heuristic so it has around 5000 nodes and 20000 edges. Then launch a visualization of it using `netwulf`, style the network so it looks nice and download (or screenshot) the figure and put it into the cell below. State explicitly what thresholds you apply and why you think they make sense. Comment on the result. Do you see anything interesting in this network or is it just a big hairball?
>
> *Hint: The longer you let the nodes **wiggle** the nicer the network will look.*

##### Answer to Ex 9.2.4:
I only examine the FB wallpost network from the past 4 days and exclude all isolated nodes (degrees less than 1). I played around in netwulf - it's a really cool tool ! A screenshot from my "*piece of art*" is shown below. 

In [ ]:
import netwulf as nw
from IPython.display import Image
G_final_plot = ans_ex_9_2_1(df_wall_posts=data,lag_days=4)
erase_nodes = [n for n,d in G_final_plot.in_degree() if d<1]
G_final_plot.remove_nodes_from(erase_nodes)
print(f'Number of nodes in network: {G_final_plot.number_of_nodes()}')
print(f'Number of edges in network: {G_final_plot.number_of_edges()}')
#nw.visualize(G_final_plot)
Image("fb_wall_post_network.png")
